# Imports

In [2]:
# Import statements
import numpy as np
from scipy.optimize import minimize
import xarray as xr
from properscoring import crps_gaussian
import pandas as pd
import matplotlib.pyplot as plt
import time
from weatherbench2.metrics import _spatial_average
from weatherbench2.visualization import set_wb2_style
from tqdm import tqdm
import concurrent.futures
import multiprocessing

In [3]:
# Random seed for reproducibility
np.random.seed(0)

In [4]:
set_wb2_style()

In [6]:
set_wb2_style()

plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 28})

# Functions

In [5]:
def np_spatial_avg(values, forecast_global):
    """
    Calculates the spatial average of a numpy.ndarray based on the global forecast coordinates.

    Parameters:
        - values (numpy.ndarray): Array of values to be averaged.
        - forecast_global (xarray.Dataset): Global forecast dataset containing longitude and latitude coordinates.

    Returns:
        float: Spatial average of the input values.
    """
    # Create a DataArray using the provided values and coordinates from forecast_global
    da = xr.DataArray(
            values,
            dims=['longitude', 'latitude'],
            coords={'longitude': forecast_global.longitude, 'latitude': forecast_global.latitude}
        )
    
    # Calculate the spatial average using the weatherbench2 function _spatial_average
    result = _spatial_average(da).values.item()
    
    return result

In [6]:
def calc_spatial_avg_metrics(df):
    
    df_mi = df.set_index(['latitude', 'longitude', 'lead_time'])
    columns_to_include = df_mi.columns
    
    # Create xarray Dataset for all specified columns
    ds_emos = xr.Dataset({
        column: (['lead_time', 'latitude', 'longitude'], df_mi[column].values.reshape((-1, len(df_mi.index.levels[0]), len(df_mi.index.levels[1]))))
        for column in columns_to_include
    }, coords={ 
        'latitude': df_mi.index.levels[0], 
        'longitude': df_mi.index.levels[1],
        'lead_time': df_mi.index.levels[2]
    })
    
    # Calculate spatial average for each metric
    spatial_avg_metrics = xr.Dataset()
    spatial_avg_metrics[df_mi.columns[1:]] = _spatial_average(ds_emos[df_mi.columns[1:]])
    
    return ds_emos, spatial_avg_metrics

In [7]:
def calc_avg_std(ds, spatial=False, s=None):
    if s is not None:
        # extract values
        emos_param_c_values = ds['emos_params_x'].str[2].values.tolist()
        emos_param_c_values = np.squeeze(np.array(emos_param_c_values), axis=-1)
        emos_param_d_values = ds['emos_params_x'].str[3].values.tolist()
        emos_param_d_values = np.squeeze(np.array(emos_param_d_values), axis=-1)
        # squeeze last axis
        local_std = np.exp(emos_param_c_values + emos_param_d_values * np.tile(np.log(s).values[:, np.newaxis, np.newaxis], (1, 32, 64)))
    else:
        # extract values
        emos_param_values = ds['emos_params_x'].str[2].values.tolist()
        # squeeze last axis and transform to sigma
        local_std = np.exp(np.squeeze(np.array(emos_param_values), axis=-1))
    
    if spatial:
        # Transpose dimensions and calculate spatial average
        local_std_transposed = np.transpose(local_std, axes=(0, 2, 1))
        local_std_avg = pd.Series(index=ds.lead_time, dtype='float64')
        for i in range(len(local_std_avg)):
            local_std_avg.iloc[i] = np_spatial_avg(local_std_transposed[i,:,:], ds)
    else:
        local_std_avg = pd.Series(np.mean(local_std, axis=(1, 2)), index=ds.lead_time)
        
    return local_std_avg

In [8]:
def emos_crps(params, ens_mean, obs, ens_sd=None, forecast_global=None):
    """
    Calculates the Continuous Ranked Probability Score (CRPS) for (ensemble) forecast using EMOS.

    Parameters:
        - params (list): List of parameters for EMOS link functions.
        - ens_mean (numpy.ndarray): Forecast ensemble mean values.
        - obs (numpy.ndarray): Observed values.
        - ens_sd (numpy.ndarray, optional): Forecast ensemble standard deviation values.
        - forecast_global (xarray.Dataset, optional): Global forecast dataset containing longitude and latitude coordinates.

    Returns:
        float: CRPS value for the given ensemble forecast and observations.
    """
    # Extract parameters
    a, b, c, *rest = params
    d = rest[0] if rest else None
    
    # Calculate location
    loc_emos = a + b * ens_mean
    
    # Calculate standard deviation  
    if ens_sd is None:
        scale_emos = np.exp(c + d * np.log(ens_mean)) if rest else np.exp(c)
    else:
        scale_emos = np.exp(c + d * np.log(ens_sd))
    
    # Calculate CRPS
    if forecast_global is None:
        # only one gridpoint
        crps = np.mean(crps_gaussian(obs, loc_emos, scale_emos))
        return crps
    else:
        # all grid points
        crps_values = crps_gaussian(obs, loc_emos, scale_emos).mean(axis=0)
        return np_spatial_avg(crps_values, forecast_global)

def emos_crps_grad(params, ens_mean, obs, ens_sd=None, forecast_global=None):
    """
    Calculates the gradient of the Continuous Ranked Probability Score (CRPS) for ensemble forecast using EMOS.

    Parameters:
        - params (list): List of parameters for EMOS link functions.
        - ens_mean (numpy.ndarray): Forecast ensemble mean values.
        - obs (numpy.ndarray): Observed values.
        - ens_sd (numpy.ndarray, optional): Forecast ensemble standard deviation values.
        - forecast_global (xarray.Dataset, optional): Global forecast dataset containing longitude and latitude coordinates.

    Returns:
        tuple: Gradient of CRPS with respect to parameters.
    """
    # Extract parameters
    a, b, c, *rest = params
    d = rest[0] if rest else None
    
    # Calculate location
    loc_emos = a + b * ens_mean
    
    # Calculate standard deviation  
    if ens_sd is None:
        scale_emos = np.exp(c + d * np.log(ens_mean)) if rest else np.exp(c)
    else:
        scale_emos = np.exp(c + d * np.log(ens_sd))
    
    # Calculate CRPS gradient
    _, grad_crps = crps_gaussian(obs, loc_emos, scale_emos, grad=True)
    
    # Extract components
    dmu, dsd = grad_crps[0], grad_crps[1]
    
    if forecast_global is None:
        # Apply chain rule for the location link function
        da = np.mean(dmu)
        db = np.mean(dmu * ens_mean)
        # Apply chain rule for the scale link function
        if not rest and ens_sd is None: 
            dc = np.mean(dsd * np.exp(c))
            return da, db, dc
        elif rest and ens_sd is None:
            dc = np.mean(dsd * np.exp(c + d * np.log(ens_mean)))
            dd = np.mean(dsd * np.exp(c + d * np.log(ens_mean)) * np.log(ens_mean))
            return da, db, dc, dd
        elif rest and ens_sd is not None:
            dc = np.mean(dsd * np.exp(c + d * np.log(ens_sd)))
            dd = np.mean(dsd * np.exp(c + d * np.log(ens_sd)) * np.log(ens_sd))
            return da, db, dc, dd
        else:
            raise ValueError('Not the right number of parameters for an ensemble forecast!')
        
    else:
        # Apply chain rule and calculate spatial average
        da_values = dmu.mean(axis=0)
        da = np_spatial_avg(da_values, forecast_global)
        
        db_values = (dmu * ens_mean).mean(axis=0)
        db = np_spatial_avg(db_values, forecast_global)
        
        if not rest and ens_sd is None: 
            dc_values = (dsd * np.exp(c)).mean(axis=0)
            dc = np_spatial_avg(dc_values, forecast_global)
            return da, db, dc
        elif rest and ens_sd is None:
            dc_values = (dsd * np.exp(c + d * np.log(ens_mean))).mean(axis=0)
            dc = np_spatial_avg(dc_values, forecast_global)
                
            dd_values = (dsd * np.exp(c + d * np.log(ens_mean)) * np.log(ens_mean)).mean(axis=0)
            dd = np_spatial_avg(dd_values, forecast_global)
            return da, db, dc, dd
        elif rest and ens_sd is not None:
            dc_values = (dsd * np.exp(c + d * np.log(ens_sd))).mean(axis=0)
            dc = np_spatial_avg(dc_values, forecast_global)
                
            dd_values = (dsd * np.exp(c + d * np.log(ens_sd)) * np.log(ens_sd)).mean(axis=0)
            dd = np_spatial_avg(dd_values, forecast_global)
            return da, db, dc, dd
        else:
            raise ValueError('Not the right number of parameters for an ensemble forecast!')

# Function for EMOS estimation
def emos_est(ens_mean, obs, par_start, ens_sd=None, forecast_global=None):
    """
    Estimates EMOS parameters using optimization based on CRPS.

    Parameters:
        - ens_mean (numpy.ndarray): Forecast ensemble mean values.
        - obs (numpy.ndarray): Observed values.
        - par_start (list): Initial values for EMOS parameters.
        - ens_sd (numpy.ndarray, optional): Forecast ensemble standard deviation values.
        - forecast_global (xarray.Dataset, optional): Global forecast dataset containing longitude and latitude coordinates.

    Returns:
        scipy.optimize.OptimizeResult: Result of the optimization, including the estimated parameters.
    """
    # Optimization using the BFGS method
    result = minimize(emos_crps, par_start, args=(ens_mean, obs, ens_sd, forecast_global), jac=emos_crps_grad)
    
    return result

# T850

## Pangu Local

In [5]:
start_time = time.time()

# Load PANGU data
pangu = xr.open_zarr('gs://weatherbench2/datasets/pangu/2018-2022_0012_64x32_equiangular_conservative.zarr')['temperature'].sel(level=850)
pangu = pangu.compute()
pangu_done_time = time.time() - start_time
print('PANGU done in {:.2f} seconds'.format(pangu_done_time))


start_time = time.time()
# Load ERA5 data
obs = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850, time=slice(pangu.time[0], pangu.time[-1]))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

PANGU done in 189.48 seconds
Obs done in 5.44 seconds


### c&d

In [7]:
result_list = []

def process_location(lat, lon, lead_time):
    try:
        pangu_subset = pangu.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

        # Split PANGU data into training and test sets
        pangu_train = xr.concat([pangu_subset.sel(time=slice(None, '2019-12-31')),
                                 pangu_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
        pangu_test = pangu_subset.sel(time=slice('2020-01-01', '2020-12-31'))

        # Load ERA5 data
        obs_train = obs.sel(latitude=lat, longitude=lon, time=pangu_train.time + lead_time)
        obs_test = obs.sel(latitude=lat, longitude=lon, time=pangu_test.time + lead_time)

        # Extract values from datasets
        pangu_train_values = pangu_train.values
        pangu_test_values = pangu_test.values
        obs_train_values = obs_train.values
        obs_test_values = obs_test.values

        # Train EMOS model
        emos_params = emos_est(pangu_train_values, obs_train_values, par_start=[0, 0, 0, 0])

        # Calculate expected value of EMOS ensemble
        emos_mu = emos_params.x[0] + emos_params.x[1] * pangu_test_values

        # Calculate CRPS, MAE, and MSE for EMOS using the expected value
        crps_emos = emos_crps(emos_params.x, pangu_test_values, obs_test_values)
        mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
        mse_emos = np.mean((emos_mu - obs_test_values)**2)

        # Calculate MAE and MSE for deterministic forecast
        mae_det = np.mean(np.abs(pangu_test_values - obs_test_values))
        mse_det = np.mean((pangu_test_values - obs_test_values)**2)

        return {
            'latitude': lat,
            'longitude': lon,
            'lead_time': lead_time,
            'emos_params_x': emos_params.x.tolist(),
            'crps_emos': crps_emos,
            'mae_emos': mae_emos,
            'mse_emos': mse_emos,
            'mae_det': mae_det,
            'mse_det': mse_det
        }
    except Exception as e:
        # Log the error
        print(f"An error occurred for lat={lat}, lon={lon}, lead_time={lead_time}: {e}")
        return None

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(pangu.prediction_timedelta.values) * len(pangu.latitude.values) * len(pangu.longitude.values)) as pbar:
    futures = []
    for lead_time in pangu.prediction_timedelta.values:
        for lat in pangu.latitude.values:
            for lon in pangu.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████| 81920/81920 [04:53<00:00, 278.91it/s]


In [9]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).to_pickle('EMOS/pangu_2020_c_d.pkl')

### just c

In [10]:
result_list = []

def process_location(lat, lon, lead_time):
    try:
        pangu_subset = pangu.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

        # Split PANGU data into training and test sets
        pangu_train = xr.concat([pangu_subset.sel(time=slice(None, '2019-12-31')),
                                 pangu_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
        pangu_test = pangu_subset.sel(time=slice('2020-01-01', '2020-12-31'))

        # Load ERA5 data
        obs_train = obs.sel(latitude=lat, longitude=lon, time=pangu_train.time + lead_time)
        obs_test = obs.sel(latitude=lat, longitude=lon, time=pangu_test.time + lead_time)

        # Extract values from datasets
        pangu_train_values = pangu_train.values
        pangu_test_values = pangu_test.values
        obs_train_values = obs_train.values
        obs_test_values = obs_test.values

        # Train EMOS model
        emos_params = emos_est(pangu_train_values, obs_train_values, par_start=[0, 0, 0])

        # Calculate expected value of EMOS ensemble
        emos_mu = emos_params.x[0] + emos_params.x[1] * pangu_test_values

        # Calculate CRPS, MAE, and MSE for EMOS using the expected value
        crps_emos = emos_crps(emos_params.x, pangu_test_values, obs_test_values)
        mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
        mse_emos = np.mean((emos_mu - obs_test_values)**2)

        # Calculate MAE and MSE for deterministic forecast
        mae_det = np.mean(np.abs(pangu_test_values - obs_test_values))
        mse_det = np.mean((pangu_test_values - obs_test_values)**2)

        return {
            'latitude': lat,
            'longitude': lon,
            'lead_time': lead_time,
            'emos_params_x': emos_params.x.tolist(),
            'crps_emos': crps_emos,
            'mae_emos': mae_emos,
            'mse_emos': mse_emos,
            'mae_det': mae_det,
            'mse_det': mse_det
        }
    except Exception as e:
        # Log the error
        print(f"An error occurred for lat={lat}, lon={lon}, lead_time={lead_time}: {e}")
        return None

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(pangu.prediction_timedelta.values) * len(pangu.latitude.values) * len(pangu.longitude.values)) as pbar:
    futures = []
    for lead_time in pangu.prediction_timedelta.values:
        for lat in pangu.latitude.values:
            for lon in pangu.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81920/81920 [04:02<00:00, 337.33it/s]


In [9]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).to_pickle('EMOS/pangu_2020_c.pkl')

## Pangu Global

In [71]:
result_list = []

def process_lead_time(lead_time):
    pangu_subset = pangu.sel(prediction_timedelta=lead_time)

    # Split pangu data into training and test sets
    pangu_train = xr.concat([pangu_subset.sel(time=slice(None, '2019-12-31')),
                             pangu_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    pangu_test = pangu_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=pangu_train.time + lead_time)
    obs_test = obs.sel(time=pangu_test.time + lead_time)

    # Extract values from datasets
    pangu_train_values = pangu_train.values
    pangu_test_values = pangu_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=pangu_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=pangu_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * pangu_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=pangu_test_values, obs=obs_test_values, forecast_global=pangu_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),pangu_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),pangu_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(pangu_test_values - obs_test_values).mean(axis=0),pangu_test)
    mse_det = np_spatial_avg(((pangu_test_values - obs_test_values)**2).mean(axis=0), pangu_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(pangu.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in pangu.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [03:23<00:00,  5.10s/it]


In [25]:
result_df.sort_values(by='lead_time').to_pickle('EMOS/pangu_2020_global_c.pkl')

## GraphCast

In [18]:
start_time = time.time()

# Load GraphCast data
graphcast = xr.concat([xr.open_zarr('gs://weatherbench2/datasets/graphcast/2018/date_range_2017-11-16_2019-02-01_12_hours-64x32_equiangular_conservative.zarr'), xr.open_zarr('gs://weatherbench2/datasets/graphcast/2020/date_range_2019-11-16_2021-02-01_12_hours-64x32_equiangular_conservative.zarr')],dim='time')['temperature'].sel(level=850)
graphcast = graphcast.compute()
graphcast_done_time = time.time() - start_time
print('graphcast done in {:.2f} seconds'.format(graphcast_done_time))


start_time = time.time()
# Load ERA5 data
obs = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850, time=slice(graphcast.time[0], graphcast.time[-1]))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

graphcast done in 209.82 seconds
Obs done in 3.80 seconds


### Global

In [19]:
result_list = []

def process_lead_time(lead_time):
    graphcast_subset = graphcast.sel(prediction_timedelta=lead_time)

    # Split graphcast data into training and test sets
    graphcast_train = xr.concat([graphcast_subset.sel(time=slice(None, '2019-12-31')),
                             graphcast_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    graphcast_test = graphcast_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=graphcast_train.time + lead_time)
    obs_test = obs.sel(time=graphcast_test.time + lead_time)

    # Extract values from datasets
    graphcast_train_values = graphcast_train.values
    graphcast_test_values = graphcast_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=graphcast_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=graphcast_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * graphcast_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=graphcast_test_values, obs=obs_test_values, forecast_global=graphcast_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),graphcast_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),graphcast_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(graphcast_test_values - obs_test_values).mean(axis=0),graphcast_test)
    mse_det = np_spatial_avg(((graphcast_test_values - obs_test_values)**2).mean(axis=0), graphcast_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(graphcast.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in graphcast.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [01:12<00:00,  1.82s/it]


In [20]:
result_df.sort_values(by='lead_time').reset_index(drop=True).to_pickle('EMOS/graphcast_2020_global_c.pkl')

### Local

In [9]:
result_list = []

def process_location(lat, lon, lead_time):
    graphcast_subset = graphcast.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

    # Split graphcast data into training and test sets
    graphcast_train = xr.concat([graphcast_subset.sel(time=slice(None, '2019-12-31')),
                             graphcast_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    graphcast_test = graphcast_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(latitude=lat, longitude=lon, time=graphcast_train.time + lead_time)
    obs_test = obs.sel(latitude=lat, longitude=lon, time=graphcast_test.time + lead_time)

    # Extract values from datasets
    graphcast_train_values = graphcast_train.values
    graphcast_test_values = graphcast_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(graphcast_train_values, obs_train_values, par_start=[0, 0, 0])

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * graphcast_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(emos_params.x, graphcast_test_values, obs_test_values)
    mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
    mse_emos = np.mean((emos_mu - obs_test_values)**2)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np.mean(np.abs(graphcast_test_values - obs_test_values))
    mse_det = np.mean((graphcast_test_values - obs_test_values)**2)

    return {
        'latitude': lat,
        'longitude': lon,
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(graphcast.prediction_timedelta.values) * len(graphcast.latitude.values) * len(graphcast.longitude.values)) as pbar:
    futures = []
    for lead_time in graphcast.prediction_timedelta.values:
        for lat in graphcast.latitude.values:
            for lon in graphcast.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81920/81920 [03:08<00:00, 435.44it/s]


In [8]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).to_pickle('EMOS/graphcast_2020_c.pkl')

## HRES

In [52]:
start_time = time.time()

# Load HRES data
hres = xr.open_zarr('gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850)
hres = hres.compute()
hres_done_time = time.time() - start_time
print('HRES done in {:.2f} seconds'.format(hres_done_time))


start_time = time.time()
# Load Analysis data
obs = xr.open_zarr('gs://weatherbench2/datasets/hres_t0/2016-2022-6h-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850, time=slice('2016-01-01', '2022-12-31'))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

HRES done in 922.06 seconds
Obs done in 8.66 seconds


### Global

In [53]:
result_list = []

def process_lead_time(lead_time):
    hres_subset = hres.sel(prediction_timedelta=lead_time)

    # Split hres data into training and test sets
    hres_train = xr.concat([hres_subset.sel(time=slice('2016-01-01', '2019-12-31')),
                             hres_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
    hres_test = hres_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=hres_train.time + lead_time)
    obs_test = obs.sel(time=hres_test.time + lead_time)

    # Extract values from datasets
    hres_train_values = hres_train.values
    hres_test_values = hres_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=hres_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=hres_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * hres_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=hres_test_values, obs=obs_test_values, forecast_global=hres_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),hres_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),hres_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(hres_test_values - obs_test_values).mean(axis=0),hres_test)
    mse_det = np_spatial_avg(((hres_test_values - obs_test_values)**2).mean(axis=0), hres_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(hres.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in hres.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [05:32<00:00,  8.11s/it]


In [24]:
result_df.sort_values(by='lead_time').to_pickle('EMOS/hres_2020_global_c.pkl')

### Local

In [54]:
result_list = []

def process_location(lat, lon, lead_time):
    hres_subset = hres.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

    # Split hres data into training and test sets
    hres_train = xr.concat([hres_subset.sel(time=slice('2016-01-01', '2019-12-31')),
                             hres_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
    hres_test = hres_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(latitude=lat, longitude=lon, time=hres_train.time + lead_time)
    obs_test = obs.sel(latitude=lat, longitude=lon, time=hres_test.time + lead_time)

    # Extract values from datasets
    hres_train_values = hres_train.values
    hres_test_values = hres_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(hres_train_values, obs_train_values, par_start=[0, 0, 0])

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * hres_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(emos_params.x, hres_test_values, obs_test_values)
    mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
    mse_emos = np.mean((emos_mu - obs_test_values)**2)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np.mean(np.abs(hres_test_values - obs_test_values))
    mse_det = np.mean((hres_test_values - obs_test_values)**2)

    return {
        'latitude': lat,
        'longitude': lon,
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(hres.prediction_timedelta.values) * len(hres.latitude.values) * len(hres.longitude.values)) as pbar:
    futures = []
    for lead_time in hres.prediction_timedelta.values:
        for lat in hres.latitude.values:
            for lon in hres.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83968/83968 [04:55<00:00, 284.45it/s]


In [11]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).to_pickle('EMOS/hres_2020_c.pkl')

## IFS ENS

In [58]:
start_time = time.time()
# Load IFS ENS data
ens = xr.open_zarr('gs://weatherbench2/datasets/ens/2018-2022-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850)
#ens = ens.compute()
ens_done_time = time.time() - start_time
print('IFS ENS done in {:.2f} seconds'.format(ens_done_time))

start_time = time.time()
# Load Analysis data
obs = xr.open_zarr('gs://weatherbench2/datasets/hres_t0/2016-2022-6h-64x32_equiangular_conservative.zarr')['temperature'].sel(level=850, time=slice('2018-01-01', '2022-12-31'))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

IFS ENS done in 1.38 seconds
Obs done in 5.22 seconds


### Global

In [8]:
result_list = []

for lead_time in tqdm(ens.prediction_timedelta.values, desc="Processing lead times"):
    
    ens_subset = ens.sel(prediction_timedelta=lead_time)
    ens_subset = ens_subset.compute()

    # Split ens data into training and test sets
    ens_train = xr.concat([ens_subset.sel(time=slice('2018-01-01', '2019-12-31')),
                             ens_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
    ens_test = ens_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=ens_train.time + lead_time)
    obs_test = obs.sel(time=ens_test.time + lead_time)

    # Extract values from datasets
    ens_train_values = ens_train.values
    ens_test_values = ens_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model 
    emos_params = emos_est(ens_mean=ens_train_values.mean(axis=1), obs=obs_train_values, par_start=[0, 0, 0, 0], ens_sd=ens_train_values.std(axis=1), forecast_global=ens_train)

    # Calculate expected value of EMOS ensemble
    ens_test_values_mean = ens_test_values.mean(axis=1)
    emos_mu = emos_params.x[0] + emos_params.x[1] * ens_test_values_mean

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=ens_test_values_mean, obs=obs_test_values, ens_sd=ens_test_values.std(axis=1), forecast_global=ens_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),ens_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),ens_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    ens_mean_mae = np_spatial_avg(np.abs(ens_test_values_mean - obs_test_values).mean(axis=0),ens_test)
    ens_mean_mse = np_spatial_avg(((ens_test_values_mean - obs_test_values)**2).mean(axis=0), ens_test)

    # Append results to the list
    result_list.append({
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'ens_mean_mae': ens_mean_mae,
        'ens_mean_mse': ens_mean_mse
    })
    
# Convert the list of dictionaries to a DataFrame and save it for each iteration
result_df = pd.DataFrame(result_list)
result_df['ens_crps'] = xr.open_dataset('Results_CLS/64x32/probabilistic/ens_vs_analysis_2020_temperature_probabilistic.nc')['temperature'].sel(metric='crps').values
result_df.to_pickle('EMOS/ens_2020_global.pkl')

Processing lead times: 100%|██████████████████| 61/61 [1:15:39<00:00, 74.42s/it]


### Local

In [ ]:
result_list = []

with tqdm(total=len(ens.prediction_timedelta.values) * len(ens.latitude.values) * len(ens.longitude.values)) as pbar:
    for lead_time in ens.prediction_timedelta.values:
        ens_sub = ens.sel(prediction_timedelta=lead_time)
        ens_sub = ens_sub.compute()
        for lat in ens.latitude.values:
            for lon in ens.longitude.values:

                ens_subset = ens_sub.sel(latitude=lat, longitude=lon)#, prediction_timedelta=lead_time)

                # Split ens data into training and test sets
                ens_train = xr.concat([ens_subset.sel(time=slice('2016-01-01', '2019-12-31')),
                                         ens_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
                ens_test = ens_subset.sel(time=slice('2020-01-01', '2020-12-31'))

                # Load ERA5 data
                obs_train = obs.sel(latitude=lat, longitude=lon, time=ens_train.time + lead_time)
                obs_test = obs.sel(latitude=lat, longitude=lon, time=ens_test.time + lead_time)

                # Extract values from datasets
                ens_train_values = ens_train.values
                ens_test_values = ens_test.values
                obs_train_values = obs_train.values
                obs_test_values = obs_test.values

                # Train EMOS model
                emos_params = emos_est(ens_mean=ens_train_values.mean(axis=1), obs=obs_train_values, par_start=[0, 0, 0, 0], ens_sd=ens_train_values.std(axis=1))

                # Calculate expected value of EMOS ensemble
                ens_test_values_mean = ens_test_values.mean(axis=1)
                emos_mu = emos_params.x[0] + emos_params.x[1] * ens_test_values_mean

                # Calculate CRPS, MAE, and MSE for EMOS using the expected value
                crps_emos = emos_crps(params=emos_params.x, ens_mean=ens_test_values_mean, obs=obs_test_values, ens_sd=ens_test_values.std(axis=1))
                mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
                mse_emos = np.mean((emos_mu - obs_test_values)**2)

                # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
                ens_mean_mae = np.mean(np.abs(ens_test_values_mean - obs_test_values))
                ens_mean_mse = np.mean((ens_test_values_mean - obs_test_values)**2)

                # Append results to the list
                result_list.append({
                    'latitude': lat,
                    'longitude': lon,
                    'lead_time': lead_time,
                    'emos_params_x': emos_params.x.tolist(),
                    'crps_emos': crps_emos,
                    'mae_emos': mae_emos,
                    'mse_emos': mse_emos,
                    'ens_mean_mae': ens_mean_mae,
                    'ens_mean_mse': ens_mean_mse
                })
                
                pbar.update(1)
    
# Convert the list of dictionaries to a DataFrame and save it for each iteration
result_df = pd.DataFrame(result_list)
#result_df['ens_crps'] = xr.open_dataset('Results_CLS/64x32/probabilistic/ens_vs_analysis_2020_temperature_probabilistic.nc')['temperature'].sel(metric='crps').values
result_df.to_pickle('EMOS/ens_2020.pkl')

# T2M

## Pangu

In [25]:
start_time = time.time()

# Load PANGU data
pangu = xr.open_zarr('gs://weatherbench2/datasets/pangu/2018-2022_0012_64x32_equiangular_conservative.zarr')['2m_temperature']
pangu = pangu.compute()
pangu_done_time = time.time() - start_time
print('PANGU done in {:.2f} seconds'.format(pangu_done_time))


start_time = time.time()
# Load ERA5 data
obs = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-64x32_equiangular_conservative.zarr')['2m_temperature'].sel(time=slice(pangu.time[0], pangu.time[-1]))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

PANGU done in 199.19 seconds
Obs done in 2.97 seconds


### Local

In [22]:
result_list = []

def process_location(lat, lon, lead_time):
    try:
        pangu_subset = pangu.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

        # Split PANGU data into training and test sets
        pangu_train = xr.concat([pangu_subset.sel(time=slice(None, '2019-12-31')),
                                 pangu_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
        pangu_test = pangu_subset.sel(time=slice('2020-01-01', '2020-12-31'))

        # Load ERA5 data
        obs_train = obs.sel(latitude=lat, longitude=lon, time=pangu_train.time + lead_time)
        obs_test = obs.sel(latitude=lat, longitude=lon, time=pangu_test.time + lead_time)

        # Extract values from datasets
        pangu_train_values = pangu_train.values
        pangu_test_values = pangu_test.values
        obs_train_values = obs_train.values
        obs_test_values = obs_test.values

        # Train EMOS model
        emos_params = emos_est(pangu_train_values, obs_train_values, par_start=[0, 0, 0])

        # Calculate expected value of EMOS ensemble
        emos_mu = emos_params.x[0] + emos_params.x[1] * pangu_test_values

        # Calculate CRPS, MAE, and MSE for EMOS using the expected value
        crps_emos = emos_crps(emos_params.x, pangu_test_values, obs_test_values)
        mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
        mse_emos = np.mean((emos_mu - obs_test_values)**2)

        # Calculate MAE and MSE for deterministic forecast
        mae_det = np.mean(np.abs(pangu_test_values - obs_test_values))
        mse_det = np.mean((pangu_test_values - obs_test_values)**2)

        return {
            'latitude': lat,
            'longitude': lon,
            'lead_time': lead_time,
            'emos_params_x': emos_params.x.tolist(),
            'crps_emos': crps_emos,
            'mae_emos': mae_emos,
            'mse_emos': mse_emos,
            'mae_det': mae_det,
            'mse_det': mse_det
        }
    except Exception as e:
        # Log the error
        print(f"An error occurred for lat={lat}, lon={lon}, lead_time={lead_time}: {e}")
        return None

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(pangu.prediction_timedelta.values) * len(pangu.latitude.values) * len(pangu.longitude.values)) as pbar:
    futures = []
    for lead_time in pangu.prediction_timedelta.values:
        for lat in pangu.latitude.values:
            for lon in pangu.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list).sort_values(by=['lead_time', 'latitude', 'longitude'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81920/81920 [04:15<00:00, 320.18it/s]


In [23]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).reset_index(drop=True).to_pickle('EMOS/T2M/pangu_2020_local.pkl')

### Global

In [27]:
result_list = []

def process_lead_time(lead_time):
    pangu_subset = pangu.sel(prediction_timedelta=lead_time)

    # Split pangu data into training and test sets
    pangu_train = xr.concat([pangu_subset.sel(time=slice(None, '2019-12-31')),
                             pangu_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    pangu_test = pangu_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=pangu_train.time + lead_time)
    obs_test = obs.sel(time=pangu_test.time + lead_time)

    # Extract values from datasets
    pangu_train_values = pangu_train.values
    pangu_test_values = pangu_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=pangu_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=pangu_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * pangu_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=pangu_test_values, obs=obs_test_values, forecast_global=pangu_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),pangu_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),pangu_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(pangu_test_values - obs_test_values).mean(axis=0),pangu_test)
    mse_det = np_spatial_avg(((pangu_test_values - obs_test_values)**2).mean(axis=0), pangu_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(pangu.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in pangu.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [04:31<00:00,  6.78s/it]


In [28]:
result_df.sort_values(by='lead_time').reset_index(drop=True).to_pickle('EMOS/T2M/pangu_2020_global.pkl')

## GraphCast

In [10]:
start_time = time.time()

# Load GraphCast data
graphcast = xr.concat([xr.open_zarr('gs://weatherbench2/datasets/graphcast/2018/date_range_2017-11-16_2019-02-01_12_hours-64x32_equiangular_conservative.zarr'), xr.open_zarr('gs://weatherbench2/datasets/graphcast/2020/date_range_2019-11-16_2021-02-01_12_hours-64x32_equiangular_conservative.zarr')],dim='time')['2m_temperature']
graphcast = graphcast.compute()
graphcast_done_time = time.time() - start_time
print('graphcast done in {:.2f} seconds'.format(graphcast_done_time))

start_time = time.time()
# Load ERA5 data
obs = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-64x32_equiangular_conservative.zarr')['2m_temperature'].sel(time=slice(graphcast.time[0], graphcast.time[-1]))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

graphcast done in 62.03 seconds
Obs done in 1.11 seconds


### Local

In [26]:
result_list = []

def process_location(lat, lon, lead_time):
    try:
        graphcast_subset = graphcast.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

        # Split graphcast data into training and test sets
        graphcast_train = xr.concat([graphcast_subset.sel(time=slice(None, '2019-12-31')),
                                 graphcast_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
        graphcast_test = graphcast_subset.sel(time=slice('2020-01-01', '2020-12-31'))

        # Load ERA5 data
        obs_train = obs.sel(latitude=lat, longitude=lon, time=graphcast_train.time + lead_time)
        obs_test = obs.sel(latitude=lat, longitude=lon, time=graphcast_test.time + lead_time)

        # Extract values from datasets
        graphcast_train_values = graphcast_train.values
        graphcast_test_values = graphcast_test.values
        obs_train_values = obs_train.values
        obs_test_values = obs_test.values

        # Train EMOS model
        emos_params = emos_est(graphcast_train_values, obs_train_values, par_start=[0, 0, 0])

        # Calculate expected value of EMOS ensemble
        emos_mu = emos_params.x[0] + emos_params.x[1] * graphcast_test_values

        # Calculate CRPS, MAE, and MSE for EMOS using the expected value
        crps_emos = emos_crps(emos_params.x, graphcast_test_values, obs_test_values)
        mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
        mse_emos = np.mean((emos_mu - obs_test_values)**2)

        # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
        mae_det = np.mean(np.abs(graphcast_test_values - obs_test_values))
        mse_det = np.mean((graphcast_test_values - obs_test_values)**2)

        return {
            'latitude': lat,
            'longitude': lon,
            'lead_time': lead_time,
            'emos_params_x': emos_params.x.tolist(),
            'crps_emos': crps_emos,
            'mae_emos': mae_emos,
            'mse_emos': mse_emos,
            'mae_det': mae_det,
            'mse_det': mse_det
        }
    except Exception as e:
        # Log the error
        print(f"An error occurred for lat={lat}, lon={lon}, lead_time={lead_time}: {e}")
        return None

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.4)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(graphcast.prediction_timedelta.values) * len(graphcast.latitude.values) * len(graphcast.longitude.values)) as pbar:
    futures = []
    for lead_time in graphcast.prediction_timedelta.values:
        for lat in graphcast.latitude.values:
            for lon in graphcast.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81920/81920 [04:04<00:00, 334.92it/s]


In [28]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).reset_index(drop=True).to_pickle('EMOS/T2M/graphcast_2020_local.pkl')

### Global

In [51]:
result_list = []

def process_lead_time(lead_time):
    graphcast_subset = graphcast.sel(prediction_timedelta=lead_time)

    # Split graphcast data into training and test sets
    graphcast_train = xr.concat([graphcast_subset.sel(time=slice(None, '2019-12-31')),
                             graphcast_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    graphcast_test = graphcast_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=graphcast_train.time + lead_time)
    obs_test = obs.sel(time=graphcast_test.time + lead_time)

    # Extract values from datasets
    graphcast_train_values = graphcast_train.values
    graphcast_test_values = graphcast_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=graphcast_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=graphcast_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * graphcast_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=graphcast_test_values, obs=obs_test_values, forecast_global=graphcast_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),graphcast_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),graphcast_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(graphcast_test_values - obs_test_values).mean(axis=0),graphcast_test)
    mse_det = np_spatial_avg(((graphcast_test_values - obs_test_values)**2).mean(axis=0), graphcast_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(graphcast.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in graphcast.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [01:25<00:00,  2.13s/it]


In [52]:
result_df.sort_values(by='lead_time').reset_index(drop=True).to_pickle('EMOS/T2M/graphcast_2020_global.pkl')

## HRES

In [36]:
start_time = time.time()

# Load HRES data
hres = xr.open_zarr('gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_conservative.zarr')['2m_temperature']
hres = hres.compute()
hres_done_time = time.time() - start_time
print('HRES done in {:.2f} seconds'.format(hres_done_time))


start_time = time.time()
# Load Analysis data
obs = xr.open_zarr('gs://weatherbench2/datasets/hres_t0/2016-2022-6h-64x32_equiangular_conservative.zarr')['2m_temperature'].sel(time=slice(hres.time[0], hres.time[-1]))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

Obs done in 2.18 seconds


### Local

In [37]:
result_list = []

def process_location(lat, lon, lead_time):
    hres_subset = hres.sel(latitude=lat, longitude=lon, prediction_timedelta=lead_time)

    # Split hres data into training and test sets
    hres_train = xr.concat([hres_subset.sel(time=slice(None, '2019-12-31')),
                             hres_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    hres_test = hres_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(latitude=lat, longitude=lon, time=hres_train.time + lead_time)
    obs_test = obs.sel(latitude=lat, longitude=lon, time=hres_test.time + lead_time)

    # Extract values from datasets
    hres_train_values = hres_train.values
    hres_test_values = hres_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(hres_train_values, obs_train_values, par_start=[0, 0, 0])

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * hres_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(emos_params.x, hres_test_values, obs_test_values)
    mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
    mse_emos = np.mean((emos_mu - obs_test_values)**2)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np.mean(np.abs(hres_test_values - obs_test_values))
    mse_det = np.mean((hres_test_values - obs_test_values)**2)

    return {
        'latitude': lat,
        'longitude': lon,
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(hres.prediction_timedelta.values) * len(hres.latitude.values) * len(hres.longitude.values)) as pbar:
    futures = []
    for lead_time in hres.prediction_timedelta.values:
        for lat in hres.latitude.values:
            for lon in hres.longitude.values:
                future = executor.submit(process_location, lat, lon, lead_time)
                futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83968/83968 [04:57<00:00, 281.96it/s]


In [39]:
result_df.sort_values(by=['lead_time', 'latitude', 'longitude']).reset_index(drop=True).to_pickle('EMOS/T2M/hres_2020_local.pkl')

### Global

In [40]:
result_list = []

def process_lead_time(lead_time):
    hres_subset = hres.sel(prediction_timedelta=lead_time)

    # Split hres data into training and test sets
    hres_train = xr.concat([hres_subset.sel(time=slice(None, '2019-12-31')),
                             hres_subset.sel(time=slice('2021-01-01', obs.time[-1] - pd.to_timedelta(lead_time)))], dim='time')
    hres_test = hres_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=hres_train.time + lead_time)
    obs_test = obs.sel(time=hres_test.time + lead_time)

    # Extract values from datasets
    hres_train_values = hres_train.values
    hres_test_values = hres_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model
    emos_params = emos_est(ens_mean=hres_train_values, obs=obs_train_values, par_start=[0, 0, 0], forecast_global=hres_train)

    # Calculate expected value of EMOS ensemble
    emos_mu = emos_params.x[0] + emos_params.x[1] * hres_test_values

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=hres_test_values, obs=obs_test_values, forecast_global=hres_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),hres_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),hres_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    mae_det = np_spatial_avg(np.abs(hres_test_values - obs_test_values).mean(axis=0),hres_test)
    mse_det = np_spatial_avg(((hres_test_values - obs_test_values)**2).mean(axis=0), hres_test)

    return {
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'mae_det': mae_det,
        'mse_det': mse_det
    }

# Automatically determine the number of workers based on CPU cores
num_workers = int(multiprocessing.cpu_count() * 0.5)

with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor, tqdm(total=len(hres.prediction_timedelta.values)) as pbar:
    futures = []
    for lead_time in hres.prediction_timedelta.values:
        future = executor.submit(process_lead_time, lead_time)
        futures.append(future)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        result_list.append(future.result())
        pbar.update(1)

executor.shutdown(wait=True)

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [05:22<00:00,  7.87s/it]


In [41]:
result_df.sort_values(by='lead_time').reset_index(drop=True).to_pickle('EMOS/T2M/hres_2020_global.pkl')

## ENS

In [9]:
start_time = time.time()
# Load IFS ENS data
ens = xr.open_zarr('gs://weatherbench2/datasets/ens/2018-2022-64x32_equiangular_conservative.zarr')['2m_temperature']
#ens = ens.compute()
ens_done_time = time.time() - start_time
print('IFS ENS done in {:.2f} seconds'.format(ens_done_time))

start_time = time.time()
# Load Analysis data
obs = xr.open_zarr('gs://weatherbench2/datasets/hres_t0/2016-2022-6h-64x32_equiangular_conservative.zarr')['2m_temperature'].sel(time=slice('2018-01-01', '2022-12-31'))
obs = obs.compute()

obs_done_time = time.time() - start_time
print('Obs done in {:.2f} seconds'.format(obs_done_time))

IFS ENS done in 12.89 seconds
Obs done in 2.31 seconds


In [10]:
ens = ens.drop_sel(time=pd.to_datetime('2019-10-17T00:00:00.000000000'))

### missing values

In [89]:
ens_train.isnull().sum().to_numpy()

array(102400)

In [87]:
ens_train.isnull().sum(dim=["number", "latitude", "longitude"]).argmax().to_numpy()

array(1308)

In [82]:
ens_train.isel(time=1308)

<xarray.DataArray '2m_temperature' (number: 50, longitude: 64, latitude: 32)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * latitude              (latitude) float64 -87.19 -81.56 ... 81.56 87.19
  * longitude             (longitude) float64 0.0 5.625 11.25 ... 348.8 354.4
  * number                (number) int32 1 2 3 4 5 6 7 ... 44 45 46 47 48 49 50
    prediction_timedelta  timedelta64[ns] 06:00:00
    time                  datetime64[ns] 2019-10-17
Attributes:
    long_name:      2 metre temperature
    short_name:     t2m
    standard_name:  unknown
    units:          K

In [74]:
ens_train.shape

(2919, 50, 64, 32)

In [84]:
50*64*32

102400

### Local

In [11]:
result_list = []

with tqdm(total=len(ens.prediction_timedelta.values) * len(ens.latitude.values) * len(ens.longitude.values)) as pbar:
    for lead_time in ens.prediction_timedelta.values:
        ens_sub = ens.sel(prediction_timedelta=lead_time)
        ens_sub = ens_sub.compute()
        for lat in ens.latitude.values:
            for lon in ens.longitude.values:

                ens_subset = ens_sub.sel(latitude=lat, longitude=lon)

                # Split ens data into training and test sets
                ens_train = xr.concat([ens_subset.sel(time=slice(None, '2019-12-31')),
                                         ens_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
                ens_test = ens_subset.sel(time=slice('2020-01-01', '2020-12-31'))

                # Load ERA5 data
                obs_train = obs.sel(latitude=lat, longitude=lon, time=ens_train.time + lead_time)
                obs_test = obs.sel(latitude=lat, longitude=lon, time=ens_test.time + lead_time)

                # Extract values from datasets
                ens_train_values = ens_train.values
                ens_test_values = ens_test.values
                obs_train_values = obs_train.values
                obs_test_values = obs_test.values

                # Train EMOS model
                emos_params = emos_est(ens_mean=ens_train_values.mean(axis=1), obs=obs_train_values, par_start=[0, 0, 0, 0], ens_sd=ens_train_values.std(axis=1))

                # Calculate expected value of EMOS ensemble
                ens_test_values_mean = ens_test_values.mean(axis=1)
                emos_mu = emos_params.x[0] + emos_params.x[1] * ens_test_values_mean

                # Calculate CRPS, MAE, and MSE for EMOS using the expected value
                crps_emos = emos_crps(params=emos_params.x, ens_mean=ens_test_values_mean, obs=obs_test_values, ens_sd=ens_test_values.std(axis=1))
                mae_emos = np.mean(np.abs(emos_mu - obs_test_values))
                mse_emos = np.mean((emos_mu - obs_test_values)**2)

                # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
                ens_mean_mae = np.mean(np.abs(ens_test_values_mean - obs_test_values))
                ens_mean_mse = np.mean((ens_test_values_mean - obs_test_values)**2)

                # Append results to the list
                result_list.append({
                    'latitude': lat,
                    'longitude': lon,
                    'lead_time': lead_time,
                    'emos_params_x': emos_params.x.tolist(),
                    'crps_emos': crps_emos,
                    'mae_emos': mae_emos,
                    'mse_emos': mse_emos,
                    'ens_mean_mae': ens_mean_mae,
                    'ens_mean_mse': ens_mean_mse
                })
                
                pbar.update(1)
    
# Convert the list of dictionaries to a DataFrame and save it for each iteration
result_df = pd.DataFrame(result_list)

  1%|██▏                                                                                                                                                                | 1656/124928 [01:57<2:14:40, 15.25it/s]/home/tbiegert/.conda/envs/wb2/lib/python3.9/site-packages/properscoring/_crps.py:16: RuntimeWarning: overflow encountered in multiply
  return _normconst * np.exp(-(x * x) / 2.0)
/home/tbiegert/.conda/envs/wb2/lib/python3.9/site-packages/properscoring/_crps.py:16: RuntimeWarning: overflow encountered in multiply
  return _normconst * np.exp(-(x * x) / 2.0)
/home/tbiegert/.conda/envs/wb2/lib/python3.9/site-packages/properscoring/_crps.py:16: RuntimeWarning: overflow encountered in multiply
  return _normconst * np.exp(-(x * x) / 2.0)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124928/124928 [2:37:02<00:00, 13.26it/s]


In [12]:
result_df.to_pickle('EMOS/T2M/ens_2020_local.pkl')

### Global

In [99]:
result_list = []

for lead_time in tqdm(ens.prediction_timedelta.values, desc="Processing lead times"):
    
    ens_subset = ens.sel(prediction_timedelta=lead_time)
    ens_subset = ens_subset.compute()

    # Split ens data into training and test sets
    ens_train = xr.concat([ens_subset.sel(time=slice(None, '2019-12-31')),
                             ens_subset.sel(time=slice('2021-01-01', obs.time[-2] - pd.to_timedelta(lead_time)))], dim='time')
    ens_test = ens_subset.sel(time=slice('2020-01-01', '2020-12-31'))

    # Load ERA5 data
    obs_train = obs.sel(time=ens_train.time + lead_time)
    obs_test = obs.sel(time=ens_test.time + lead_time)

    # Extract values from datasets
    ens_train_values = ens_train.values
    ens_test_values = ens_test.values
    obs_train_values = obs_train.values
    obs_test_values = obs_test.values

    # Train EMOS model 
    emos_params = emos_est(ens_mean=ens_train_values.mean(axis=1), obs=obs_train_values, par_start=[0, 0, 0, 0], ens_sd=ens_train_values.std(axis=1), forecast_global=ens_train)

    # Calculate expected value of EMOS ensemble
    ens_test_values_mean = ens_test_values.mean(axis=1)
    emos_mu = emos_params.x[0] + emos_params.x[1] * ens_test_values_mean

    # Calculate CRPS, MAE, and MSE for EMOS using the expected value
    crps_emos = emos_crps(params=emos_params.x, ens_mean=ens_test_values_mean, obs=obs_test_values, ens_sd=ens_test_values.std(axis=1), forecast_global=ens_test)
    mae_emos = np_spatial_avg(np.abs(emos_mu - obs_test_values).mean(axis=0),ens_test)
    mse_emos = np_spatial_avg(((emos_mu - obs_test_values)**2).mean(axis=0),ens_test)

    # Calculate MAE and MSE for deterministic forecast (replace this line with your deterministic forecast calculation)
    ens_mean_mae = np_spatial_avg(np.abs(ens_test_values_mean - obs_test_values).mean(axis=0),ens_test)
    ens_mean_mse = np_spatial_avg(((ens_test_values_mean - obs_test_values)**2).mean(axis=0), ens_test)

    # Append results to the list
    result_list.append({
        'lead_time': lead_time,
        'emos_params_x': emos_params.x.tolist(),
        'crps_emos': crps_emos,
        'mae_emos': mae_emos,
        'mse_emos': mse_emos,
        'ens_mean_mae': ens_mean_mae,
        'ens_mean_mse': ens_mean_mse
    })
    
# Convert the list of dictionaries to a DataFrame and save it for each iteration
result_df = pd.DataFrame(result_list)

Processing lead times: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [1:07:23<00:00, 66.29s/it]


In [100]:
result_df.to_pickle('EMOS/T2M/ens_2020_global.pkl')